In [1]:
import numpy as np  
import pandas as pd  
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
!pip install xgboost
from xgboost import XGBClassifier
import pyarrow.feather as feather
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import balanced_accuracy_score, confusion_matrix,classification_report
from sklearn.model_selection import train_test_split
import shap
# Suppress warnings 
# (sometimes you might want to ignore warnings, that's how you can achieve this)
import warnings
import shap
warnings.filterwarnings('ignore')
RSEED= 42

In [2]:
export_df = feather.read_feather("../data/cleaned_data.feather")

In [3]:
export_df['lenght']=[row.astype(int) for row in export_df['lenght']]

In [4]:
# Function to split the dataset 
def splitdataset(df):
    y=export_df["interesting_message"]
    X=export_df.drop("interesting_message",axis=1)
    X_train, X_test, y_train, y_test = train_test_split(X,y, stratify = y, test_size = 0.2,random_state = 42)
    print(y_train.dtypes)
    return X_train, X_test, y_train, y_test

In [5]:
def train_xgb(X_train, X_test, y_train): 
    # Creating the classifier object 
    xgb_class = XGBClassifier(use_label_encoder=False,eval_metric= "logloss")
    xgb_parametering= {'scale_pos_weight': [1,2,4]}
    #1,2,4
    cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
    xgb_grid = GridSearchCV(estimator=xgb_class, param_grid=xgb_parametering, cv=cv,scoring='balanced_accuracy',
    verbose=10, n_jobs=-1)
    grid_result=xgb_grid.fit(X_train,y_train)
    best_model=grid_result.best_estimator_
    # report the best configuration
    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
    return best_model

In [6]:
# Function to make predictions 
def prediction(X_test, model): 
    y_pred = model.predict(X_test)
    return y_pred 

In [7]:
# Function to calculate accuracy 
def class_metrics(y_test, y_pred): 
    accuracy = balanced_accuracy_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred).round()
    print("Predicted values:\n", y_pred) 
    print("Confusion Matrix: \n", cm) 
    print("Balanced Accuracy: %.4f%%" % (accuracy * 100.0))
    print("Report : \n", classification_report(y_test, y_pred))
    return cm, accuracy

In [8]:
def plot_importance (model,X_train):
    explainer=shap.TreeExplainer(model,data=X_train)
    shap_values = explainer.shap_values(X_train)
    vals= np.abs(shap_values).mean(0)
    feature_importance = pd.DataFrame(list(zip(X_train.columns, sum(vals))), columns=['col_name','feature_importance_vals'])
    ordered_weights= feature_importance.sort_values(by=['feature_importance_vals'], ascending=False,inplace=True)
    shap.summary_plot(shap_values, X_train, plot_type="bar")
    
    return ordered_weights

In [9]:
# Driver code 
def main(): 
    # Building Phase 
    X_train, X_test, y_train, y_test = splitdataset(export_df) 
    basemodel = train_xgb(X_train, X_test, y_train)
    # Operational Phase 
    print("-----"*15)
    print("Results:\n")
    # Prediction
    y_pred = prediction(X_test, basemodel) 
    cm,accuracy=class_metrics(y_test, y_pred)
    return basemodel,y_pred,cm,accuracy,X_train

In [10]:
basemodel,y_pred,cm,accuracy,X_train=main()

int64
Fitting 15 folds for each of 1 candidates, totalling 15 fits
[CV 7/15; 1/1] START scale_pos_weight=1.........................................
[CV 1/15; 1/1] START scale_pos_weight=1.........................................
[CV 5/15; 1/1] START scale_pos_weight=1.........................................
[CV 8/15; 1/1] START scale_pos_weight=1.........................................
[CV 4/15; 1/1] START scale_pos_weight=1.........................................
[CV 2/15; 1/1] START scale_pos_weight=1.........................................
[CV 6/15; 1/1] START scale_pos_weight=1.........................................
[CV 3/15; 1/1] START scale_pos_weight=1.........................................
[CV 9/15; 1/1] START scale_pos_weight=1.........................................


KeyboardInterrupt: 

In [ ]:
ordered_weights=plot_importance (basemodel,X_train)

In [ ]:
f = open('ml-log_xgboost.txt', 'a')
f.write('Base model: XGBoost\n Predicted values:\n {}\n Confusion Matrix:\n {}\n Balanced Accuracy:\n {} \n Model: {}'.format(y_pred,cm,accuracy,basemodel))
f.close()